In [17]:
import numpy as np

In [36]:
import array
import random

from deap import base
from deap import creator
from deap import tools

creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", array.array, typecode="d",
               fitness=creator.FitnessMin, strategy=None)
creator.create("Strategy", array.array, typecode="d")

def initES(icls, scls, size, imin, imax, smin, smax):
    ind = icls(random.uniform(imin, imax) for _ in range(size))
    ind.strategy = scls(random.uniform(smin, smax) for _ in range(size))
    return ind

IND_SIZE = 10
MIN_VALUE, MAX_VALUE = -5., 5.
MIN_STRAT, MAX_STRAT = -1., 1. 

toolbox = base.Toolbox()
toolbox.register("individual", initES, creator.Individual,
                 creator.Strategy, IND_SIZE, MIN_VALUE, MAX_VALUE, MIN_STRAT, 
                 MAX_STRAT)

/Users/nakatani/.pyenv/versions/anaconda3-5.3.0/envs/m1/lib/python3.7/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/Users/nakatani/.pyenv/versions/anaconda3-5.3.0/envs/m1/lib/python3.7/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


In [21]:
import random
from deap import creator, base, tools, algorithms

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()

toolbox.register("attr_bool", random.randint, 0, 1)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=100)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

def evalOneMax(individual):
    return sum(individual),

toolbox.register("evaluate", evalOneMax)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)

population = toolbox.population(n=300)

NGEN=40
for gen in range(NGEN):
    offspring = algorithms.varAnd(population, toolbox, cxpb=0.5, mutpb=0.1)
    fits = toolbox.map(toolbox.evaluate, offspring)
    for fit, ind in zip(fits, offspring):
        ind.fitness.values = fit
    population = toolbox.select(offspring, k=len(population))
top10 = tools.selBest(population, k=10)

In [33]:
print(f'population: {np.array(population).shape}')
print(f'top10: {np.array(top10).shape}')
print(f'individual: {np.array(tools.initRepeat(creator.Individual, toolbox.attr_bool, n=100)).shape}')
print(f'offspring: {np.array(offspring).shape}')


population: (300, 100)
top10: (10, 100)
individual: (100,)
offspring: (300, 100)


In [35]:
toolbox

## ↓これが最終のソースコード

In [5]:
'''
Created on 2022/07/07

@author: mori
'''
import numpy as np
import random
import copy

class Individual:
    def __init__(self,size=10):
        self._size=size
        self._chromosome = np.array(np.random.randint(0,2,size))     
       
    def getSize(self):
        return self._size
    def getChromosome(self):
        return self._chromosome
    def getGene(self,i):
        return self._chromosome[i]
    def setGene(self,i,value):
        self._chromosome[i]=value
    def __str__(self):
        return str(self._chromosome)
    def __len__(self): #same as self.getSize()
        return self._chromosome.size
    def fitness(self):
        return sum(self._chromosome) #OneMax problem

        
def mutation(pop):
    mutation_rate=1/len(pop[0]) #common setting
    for indiv in pop:
        for i in range(len(indiv)):
            rnd=np.random.rand()
            if(rnd<mutation_rate):
                if(indiv.getGene(i)==1):
                    indiv.setGene(i,0)
                else:
                    indiv.setGene(i,1)

def crossover(pop):
    idx = np.arange(len(pop))
    random.shuffle(idx)
    itr = iter(idx)
    for i, j in zip(itr, itr):
        rand = np.random.randint(10)
#         print('be', pop[i], pop[j])
        for k in range(rand):
            pop_i_old = Individual(20)
            for id_, val in enumerate(pop[0].getChromosome()):
                pop_i_old.setGene(id_, val)
            pop[i].setGene(k, pop[j].getGene(k))
            pop[j].setGene(k, pop_i_old.getGene(k))
#         print('af', pop[i], pop[j])

    
def selection(pop):
    sorted_pop_n_idx = (sorted([(indiv.fitness(), i) for i, indiv in enumerate(pop)], reverse=True))
    sorted_pop = [pop[idx] for _, idx in sorted_pop_n_idx]
    for id_, val in enumerate(pop[0].getChromosome()):
        sorted_pop[-1].setGene(id_, val)
    pop = sorted_pop

def getElite(pop):
    elite=pop[0]
    bestFitness=pop[0].fitness()
    for indiv in pop[1:]:
        if(indiv.fitness()>bestFitness):
            elite=indiv
            bestFitness=indiv.fitness()
    return elite
    
def printPop(pop):
    for indiv in pop:
        print(indiv,indiv.fitness())
         
pop_size=10
generation_size=100
pop=[Individual(20) for i in range(pop_size)]
elite = pop[0]
for t in range(generation_size):
    print(elite, elite.fitness())
    mutation(pop)
    crossover(pop)
    selection(pop)
    if elite.fitness() < getElite(pop).fitness():
#         print('be', elite_.fitness())
        elite = copy.deepcopy(getElite(pop))
#         print('af', getElite(pop).fitness())
        

#     print(elite,elite.fitness())

[1 0 1 1 0 1 0 0 1 1 0 1 1 0 0 0 1 0 0 1] 10
[1 1 0 1 0 1 0 0 0 1 1 1 0 1 1 1 1 0 1 1] 13
[1 1 0 1 0 1 0 0 0 1 1 1 0 1 1 1 1 0 1 1] 13
[1 1 0 1 1 1 1 0 1 1 1 1 1 0 0 0 1 0 1 1] 14
[1 1 0 1 1 1 1 0 1 1 1 1 1 0 0 0 1 0 1 1] 14
[1 1 0 1 1 1 1 0 1 1 1 1 1 0 0 0 1 0 1 1] 14
[1 0 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 0 0 1] 15
[1 0 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 0 0 1] 15
[1 0 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 0 0 1] 15
[1 0 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 0 0 1] 15
[1 0 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 0 0 1] 15
[1 0 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 0 0 1] 15
[1 0 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 0 0 1] 15
[1 0 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 0 0 1] 15
[1 0 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 0 0 1] 15
[1 0 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 0 0 1] 15
[1 0 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 0 0 1] 15
[1 0 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 0 0 1] 15
[0 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 0] 16
[0 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 0] 16
[0 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 0] 16
[0 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 0] 16
[0 1 1 1 1

In [159]:
list(zip(pop, ['\n']*len(pop)))

[(<__main__.Individual at 0x7fd0e0e89d50>, '\n'),
 (<__main__.Individual at 0x7fd0e0e88590>, '\n'),
 (<__main__.Individual at 0x7fd0e0e88f90>, '\n'),
 (<__main__.Individual at 0x7fd0e0e90e50>, '\n'),
 (<__main__.Individual at 0x7fd0e0e907d0>, '\n'),
 (<__main__.Individual at 0x7fd0e0e90690>, '\n'),
 (<__main__.Individual at 0x7fd0e0e90810>, '\n'),
 (<__main__.Individual at 0x7fd0e0e90d10>, '\n'),
 (<__main__.Individual at 0x7fd0e0e90c90>, '\n'),
 (<__main__.Individual at 0x7fd0e0e901d0>, '\n')]

In [155]:
'''
Created on 2022/07/07

@author: mori
'''
import numpy as np
import random

class Individual:
    def __init__(self,size=10):
        self._size=size
        self._chromosome = np.array(np.random.randint(0,2,size))     
       
    def getSize(self):
        return self._size
    def getChromosome(self):
        return self._chromosome
    def getGene(self,i):
        return self._chromosome[i]
    def setGene(self,i,value):
        self._chromosome[i]=value
    def __str__(self):
        return str(self._chromosome)
    def __len__(self): #same as self.getSize()
        return self._chromosome.size
    def fitness(self):
        return sum(self._chromosome) #OneMax problem

        
def mutation(pop):
    mutation_rate=1/len(pop[0]) #common setting
    for indiv in pop:
        for i in range(len(indiv)):
            rnd=np.random.rand()
            if(rnd<mutation_rate):
                if(indiv.getGene(i)==1):
                    indiv.setGene(i,0)
                else:
                    indiv.setGene(i,1)

def crossover(pop):
    idx = np.arange(len(pop))
    random.shuffle(idx)
    itr = iter(idx)
    for i, j in zip(itr, itr):
        rand = np.random.randint(10)
#         print('be', pop[i], pop[j])
        for k in range(rand):
            pop_i_old = Individual(20)
            for id_, val in enumerate(pop[0].getChromosome()):
                pop_i_old.setGene(id_, val)
            pop[i].setGene(k, pop[j].getGene(k))
            pop[j].setGene(k, pop_i_old.getGene(k))
#         print('af', pop[i], pop[j])

    
def selection(pop):
    sorted_pop_n_idx = (sorted([(indiv.fitness(), i) for i, indiv in enumerate(pop)], reverse=True))
    sorted_pop = [pop[idx] for _, idx in sorted_pop_n_idx]
    for id_, val in enumerate(pop[0].getChromosome()):
        sorted_pop[-1].setGene(id_, val)
    pop = sorted_pop

def getElite(pop):
    elite=pop[0]
    bestFitness=pop[0].fitness()
    for indiv in pop[1:]:
        if(indiv.fitness()>bestFitness):
            elite=indiv
            bestFitness=indiv.fitness()
    return elite
    
def printPop(pop):
    for indiv in pop:
        print(indiv,indiv.fitness())
         
pop_size=10
generation_size=100
pop=[Individual(20) for i in range(pop_size)]
for t in range(generation_size):
    mutation(pop)
    crossover(pop)
    selection(pop)
    
    elite = getElite(pop)
    print(elite,elite.fitness())

[0 1 1 1 0 0 1 0 1 1 1 0 0 1 1 1 0 1 1 0] 12


In [65]:
# def selection(pop):
#     print("selection")

    '''
Created on 2022/07/07

@author: mori
'''
import numpy as np
import random

class Individual:
    def __init__(self,size=10):
        self._size=size
        self._chromosome = np.array(np.random.randint(0,2,size))     
       
    def getSize(self):
        return self._size
    def getChromosome(self):
        return self._chromosome
    def getGene(self,i):
        return self._chromosome[i]
    def setGene(self,i,value):
        self._chromosome[i]=value
    def __str__(self):
        return str(self._chromosome)
    def __len__(self): #same as self.getSize()
        return self._chromosome.size
    def fitness(self):
        return sum(self._chromosome) #OneMax problem

        
def mutation(pop):
    mutation_rate=1/len(pop[0]) #common setting
    for indiv in pop:
        for i in range(len(indiv)):
            rnd=np.random.rand()
            if(rnd<mutation_rate):
                if(indiv.getGene(i)==1):
                    indiv.setGene(i,0)
                else:
                    indiv.setGene(i,1)

def crossover(pop):
    idx = np.arange(len(pop))
    random.shuffle(idx)
    itr = iter(idx)
    for i, j in zip(itr, itr):
        rand = np.random.randint(10)
#         print('be', pop[i], pop[j])
        for k in range(rand):
            pop_i_old = Individual(20)
            for id_, val in enumerate(pop[0].getChromosome()):
                pop_i_old.setGene(id_, val)
            pop[i].setGene(k, pop[j].getGene(k))
            pop[j].setGene(k, pop_i_old.getGene(k))
#         print('af', pop[i], pop[j])

    
def selection(pop):
    sorted_pop_n_idx = (sorted([(indiv.fitness(), i) for i, indiv in enumerate(pop)], reverse=True))
    sorted_pop = [pop[idx] for _, idx in sorted_pop_n_idx]
    for id_, val in enumerate(pop[0].getChromosome()):
        sorted_pop[-1].setGene(id_, val)
    pop = sorted_pop

def getElite(pop):
    elite=pop[0]
    bestFitness=pop[0].fitness()
    for indiv in pop[1:]:
        if(indiv.fitness()>bestFitness):
            elite=indiv
            bestFitness=indiv.fitness()
    return elite
    
def printPop(pop):
    for indiv in pop:
        print(indiv,indiv.fitness())
         
pop_size=10
generation_size=100
pop=[Individual(20) for i in range(pop_size)]
elite_ = pop[0]
for t in range(generation_size):
    mutation(pop)
    crossover(pop)
    selection(pop)
    if elite_.fitness() < getElite(pop).fitness():
        print('be', elite_.fitness())
        elite_ = getElite(pop)
        print('af', getElite(pop).fitness())
        

#     print(elite,elite.fitness())

SyntaxError: unexpected EOF while parsing (<ipython-input-65-0094424ca964>, line 4)

In [75]:
pop = sorted_pop[:len(sorted_pop)/2]

NameError: name 'sorted_pop' is not defined

In [74]:
pop

In [78]:
sorted_pop_n_idx = (sorted([(indiv.fitness(), i) for i, indiv in enumerate(pop)], reverse=True))
sorted_pop = [pop[idx] for _, idx in sorted_pop_n_idx]
pop_ = sorted_pop[:len(sorted_pop)//2]


In [94]:
sorted_pop_n_idx = (sorted([(indiv.fitness(), i) for i, indiv in enumerate(pop)], reverse=True))
sorted_pop = [pop[idx] for _, idx in sorted_pop_n_idx]
sorted_pop[-1] = sorted_pop[0]
pop_ = sorted_pop

In [96]:
len(pop_)

10

In [109]:
idx = np.arange(len(pop))
random.shuffle(idx)
itr = iter(idx)
for i, j in zip(itr, itr):
    rand = np.random.randint(10)
    for k in range(10-rand):
        pop[i].setGene(k+rand, pop[j].getGene(k+rand))

TypeError: 'float' object cannot be interpreted as an integer

In [108]:
pop[j]

In [105]:
[ind.fitness() for ind in pop_]

[15, 14, 12, 12, 12, 11, 11, 10, 9, 15]

In [ ]:
idx = np.arange(len(pop))
random.shuffle(idx)
itr = iter(idx)
for i, j in zip(itr, itr):
    rand = np.random.randint(10)
    for k in range(10-rand):
        pop_i_old = individual(1)
        for id_, val in enumerate(pop[0].getChromosome()):
            pop_i_old.setGene(id_, val)
        pop[i].setGene(k+rand, pop[j].getGene(k+rand))
        pop[j].setGene(k+rand, pop_i_old.getGene(k+rand))

In [120]:
pop_i_old = pop[i].copy()

AttributeError: 'Individual' object has no attribute 'copy'

array([1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0])